#### 단어 사전-RNN

In [ ]:
from tensorflow.python.framework import ops
import pprint
import tensorflow as tf
import numpy as np
ops.reset_default_graph()
pp = pprint.PrettyPrinter(indent=4)
sess = tf.InteractiveSession()

In [ ]:
# one-hot-encoding
h = [1,0,0,0]
e = [0,1,0,0]
l = [0,0,1,0]
o = [0,0,0,1]

In [ ]:
with tf.variable_scope('five_sequences') as scope:
    hidden_size = 2
    cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
    x_data = np.array([[h,e,l,l,o]], dtype=np.float32) #5x4
    print(x_data.shape)
    pp.pprint(x_data)
    
    # embedding한다: vector화 한다 (문자를 수치화)
    outputs, states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32) # 5x2
    
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())
    print(outputs.shape)
    print(states.shape)
    
    pp.pprint(states.eval())

#### 단어 맞추기

In [ ]:
char_arr = ['a', 'b', 'c', 'd', 'e', 'f', 'g',
            'h', 'i', 'j', 'k', 'l', 'm', 'n',
            'o', 'p', 'q', 'r', 's', 't', 'u',
            'v', 'w', 'x', 'y', 'z']

In [ ]:
# Map 타입
num_dic = {n: i for i,n in enumerate(char_arr)}
dic_len = len(num_dic)
seq_data = ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']

In [ ]:
def make_batch(seq_data):
    input_batch = []
    target_batch = []
    for seq in seq_data:
        input = [num_dic[n] for n in seq[:-1]]
        target = num_dic[seq[-1]]
        
        # .eye 단위행렬
        input_batch.append(np.eye(dic_len)[input])
        target_batch.append(target_batch)
        
    return input_batch, target_batch

In [ ]:
# architecture
learning_rate = 0.01
n_hidden = 128
total_epoch = 30
n_step = 3
n_input = n_class = dic_len


In [ ]:
X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.int32, [None])

In [ ]:
W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

In [ ]:
cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5, seed=10)
cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)

multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])


In [ ]:
outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)

In [ ]:
outputs = tf.transpose(outputs, [1,0,2])
outputs = outputs[-1]

In [ ]:
model = tf.matmul(outputs, W) + b
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, target_batch = make_batch(seq_data)
for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost], feed_dict={X:input_batch, Y: target_batch})
    print('Epoh:' '%04d' %(epoch+1), 'cost=', '{:.6f}'.format(loss))

In [ ]:
prediction = tf.cast(tf.argmax(model,1), tf.int32)
prediction_check = tf.equal(prediciton, Y)

accuracy = tf.reduce_mean(tf.cast(prediction_check, tf.float32))
input_batch, target_batch = make_batch(seq_data)
predict, accuracy_val = sess.run([prediction, accuracy], feed_dict={X:input_batch, Y:target_batch})

In [ ]:
predict_words = []
for idx, val in enumerate(seq_data):
    last_char = char_arr[predict[idx]]
    predict_words.append(val[:3]+last_char)
    
print('##########예측결과 ###########')
print('입력값:', [w[:3]+''for w in seq_data])
print('예측값:', predict_words)
print('정확도:', accuracy_val)